Toy case, 

$X\sim N(\mu,\Sigma)$, $f(X)=[1,0]^T X+b$, $g(X)=BX+b$. Solve the following optimization problem 

$$ \min_{B,a,b} \text{HSIC}(f(X),g(X)) - (B - [1,0])^2 $$

In [ ]:
from IPython.core.getipython import get_ipython
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')


In [ ]:
import numpy as np
import numpy.random as npr
import scipy.stats as stats

import matplotlib.pyplot as plt
from jaxkern import (
    rbf_kernel, linear_kernel, estimate_sigma_median, hsic, mmd)

import jax
from jax.experimental import optimizers
from jax import grad, random
import jax.numpy as jnp

In [ ]:
def plt_kde(X):
    fig, ax = plt.subplots()
    fig.set_size_inches(6,6)
    lim = np.min(X)-1, np.max(X)+1
    XX,YY = np.meshgrid(
        np.linspace(lim[0], lim[1], 100),
        np.linspace(lim[0], lim[1], 100))
    XY = np.vstack([XX.ravel(), YY.ravel()])
    kernel = stats.gaussian_kde(X.T)
    Z = kernel(XY).reshape(XX.shape)
    # ax.scatter(X[:,0], X[:,1], c='k', s=3)
    ax.imshow(Z, cmap='Blues',
              extent=[lim[0], lim[1], lim[0], lim[1]])
    ax.set_xlim([lim[0], lim[1]])
    ax.set_ylim([lim[0], lim[1]])
    return fig, ax

In [ ]:
n = 100
d = 2
lr = .01
num_steps = 100
batch_size = 128

npr.seed(0)
mu  = np.array([0,0])
cov = np.array([[1,0], [0,1]])
Xdist = stats.multivariate_normal(mu, cov)
X = Xdist.rvs(size=(n,))
X = jax.device_put(X)

sigma = estimate_sigma_median(X)
gamma = 1/(2*(sigma**2))
print(f'sigma={sigma}')
print(f'gamma={gamma}')

def kernel(X,Y):
    return rbf_kernel(X,Y,gamma=gamma)

A = jnp.array([[1,0]], dtype=jnp.float32)
params = {
    'B': jnp.array([[.5,.5]], dtype=jnp.float32),
#     'a': random.normal(key, (1,)),
#     'b': random.normal(key, (1,)),
}

def f(params, X):
    return jnp.dot(A,X)
def g(params, X):
    return jnp.dot(params['B'],X)
f = jax.vmap(f, (None, 0), 0)
g = jax.vmap(g, (None, 0), 0)

def loss_fn(params, X):
    fX = f(params, X)
    gX = g(params, X)
    loss = hsic(fX, gX, kernel, kernel) + cosine_sim(A.T, params['B'].T) + jnp.linalg.norm(params['B'], ord=2)
    return loss


opt_init, opt_update, get_params = optimizers.sgd(lr)
opt_state = opt_init(params)

print("\nStarting training...")
for i in range(num_steps):
    start_time = time.time()
    params = get_params(opt_state)
    
    fX = f(params, X)
    gX = g(params, X)
    fgX = jnp.hstack([fX, gX])
    
    l = loss_fn(params, X)
    grd = grad(loss_fn)(get_params(opt_state), X)
    opt_state = opt_update(i, grd, opt_state)
    
    epoch_time = time.time() - start_time
    
    
    
    
    if i%5 == 0:
        print(f'[{i:3}] time={epoch_time:4.4f}\t loss={l:5.8f}\t A^TB={jnp.dot(A,params["B"].T)} B={params["B"]}')
        
        plt.scatter(fgX[:,0], fgX[:,1], label=i)
        plt.xlim((-5,5))
        plt.ylim((-5,5))
        
        
        
    
plt.legend()
    
    


In [ ]:

foo = jnp.array([[1,2],[3,4]])
fill_diagonal(foo, 0)

In [ ]:
B = params['B']
jax.numpy.linalg.norm(B-A, ord=2)